In [5]:
from transformers import pipeline,PegasusTokenizer,PegasusForConditionalGeneration,T5Tokenizer,T5ForConditionalGeneration

In [6]:

concatinated_texts = """Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in
our lives. Hope you too, to enjoy it as my deepest wishes.
Thank your message to show our words to the doctor, as his next contract checking, to all of us.
I got this message to see the approved message. In fact, I have received the message from the
professor, to show me, this, a couple of days ago. I am very appreciated the full support of the
professor, for our Springer proceedings publication
“During our final discuss, I told him about the new submission — the one we were waiting since
last autumn, but the updates was confusing as it not included the full feedback from reviewer or
maybe editor?Anyway, I believe the team, although bit delay and less communication at recent days, they really
tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance
and efforts until the Springer link came finally last week, I think.
Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before
he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so.
Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future
targets”
"""


In [7]:
# pipeline-bart-large για περίληψη,χρησιμοποιωντας cpu
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer(concatinated_texts, max_length=200, min_length=20, do_sample=False)

print("Περίληψη:", summary[0]['summary_text'])

Device set to use cpu


Περίληψη: The author received the approved version of the manuscript a few days ago. The manuscript was submitted to Springer for publication last week. The author says he is grateful for the full support of the Springer team.


In [8]:
model_name = "google/pegasus-large"

# Φορτώνουμε explicit τον tokenizer και το μοντέλο
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenize το input
inputs = tokenizer(concatinated_texts, return_tensors="pt", truncation=True)

# Παράγουμε την ανακατασκευή / σύνοψη
summary_ids = model.generate(
    **inputs,
    max_length=100,
    min_length=30,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Μετατρέπουμε τα tokens πίσω σε κείμενο
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Thank your message to show our words to the doctor, as his next contract checking, to all of us. I am very appreciated the full support of the professor, for our Springer proceedings publication “During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the complete feedback from reviewer or maybe editor?Anyway,I believe the team, although bit delay and less communication at recent days, they really


In [12]:
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)



# Προετοιμασία του input για T5 (prefix "paraphrase: " για παραφράσεις)
input_text = "paraphrase: " + concatinated_texts
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Δημιουργία ανακατασκευής / παραφράσεων
output = model.generate(
    **inputs,
    max_length=250,
    min_length=30,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Αποκωδικοποίηση παραφρασμένων κειμένων
final_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(final_output)


Falkenstein: today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great . I am very grateful, all of us, for the acceptance and efforts until the Springer link came finally last week, I think.
